In [ ]:
import os
import openai
import faiss
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Set OpenAI API Key
openai.api_key = "your_openai_api_key"

# Load and process research paper (PDF)
def load_paper(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    return documents

# Text Chunking
def chunk_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(documents)
    return chunks

# Embed and Store in FAISS
def create_faiss_index(chunks):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(chunks, embeddings)
    return vector_store

# Q&A System
def answer_question(query, vector_store):
    retriever = vector_store.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(model_name="gpt-4"), retriever=retriever)
    return qa_chain.run(query)

# Main Execution
pdf_path = "research_paper.pdf"  # Change this to your PDF file
documents = load_paper(pdf_path)
chunks = chunk_text(documents)
vector_store = create_faiss_index(chunks)

# Ask a question
query = "What are the main findings of the paper?"
answer = answer_question(query, vector_store)

print("Answer:", answer)
